In [1]:
%matplotlib inline
from __future__ import division
import matplotlib
import numpy as np
from pylab import *
import os
matplotlib.rcParams.update({"axes.formatter.limits": (-4,4)})
plotStyles={"markersize":12,"markeredgewidth":3.0,"linewidth":3.0}
stepStyles={"markersize":12,"markeredgewidth":3.0,"linewidth":3.0,"where":"post"}
np.seterr(divide='ignore',invalid='ignore')
pass

### Setup the notebook.

In [2]:
import h5py
import math
testNames=["yeast_calcium_homeostasis"]
test_names_bash_list=" ".join(testNames)
def isclose(a, b, rel_tol=1e-09, abs_tol=0.0):
    return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

In [3]:
%%bash
rm -rf tmp && mkdir tmp

### Execute the SBML imports.

In [4]:
%%bash -s "$test_names_bash_list"
for testName in $1; do
    inputFilename=${testName}.xml
    outputFilename=tmp/${testName}.lm
    rm -f ${outputFilename}*
    if [[ ${testName} == yeast_calcium_homeostasis ]]; then
        lm_sbml_import --verbose --verbose --copasi --ignore-variable-parameters ${outputFilename} ${inputFilename} 2>&1 > ${outputFilename}.log
    else
        echo "Invalid test name: ${testName}"
        continue
    fi
done;
echo "Finished."

Finished.


### Define the tests.

In [5]:
def test_dimerization_reversible(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if nr != 2: raise Exception("numberReactions: incorrect value")
    if ns != 2: raise Exception("numberSpecies: incorrect value")
    if D.shape != (2,2): raise Exception("D: incorrect shape",D.shape)
    if D[0,0] != 1: raise Exception("D: incorrect value")
    if D[0,1] != 0: raise Exception("D: incorrect value")
    if D[1,0] != 0: raise Exception("D: incorrect value")
    if D[1,1] != 1: raise Exception("D: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if C[0] != 2000: raise Exception("C: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],0.95,1e-4): raise Exception("K: incorrect value")
    for i in range(1,10):
        if not math.isnan(K[0,i]): raise Exception("K: incorrect nan value")
    if not isclose(K[1,0],10.0,1e-4): raise Exception("K: incorrect value")
    for i in range(1,10):
        if not math.isnan(K[1,i]): raise Exception("K: incorrect nan value")
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if R[0] != 3: raise Exception("R: incorrect value")
    if R[1] != 1: raise Exception("R: incorrect value")
    if S.shape != (2,2): raise Exception("S: incorrect shape")
    if S[0,0] != -2: raise Exception("S: incorrect value")
    if S[0,1] != 2: raise Exception("S: incorrect value")
    if S[1,0] != 1: raise Exception("S: incorrect value")
    if S[1,1] != -1: raise Exception("S: incorrect value")
    fp.close()

In [6]:
def test_first_order_birth_death(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if nr != 2: raise Exception("numberReactions: incorrect value")
    if ns != 1: raise Exception("numberSpecies: incorrect value")
    if D.shape != (1,2): raise Exception("D: incorrect shape",D.shape)
    if D[0,0] != 0: raise Exception("D: incorrect value")
    if D[0,1] != 1: raise Exception("D: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if C[0] != 1000: raise Exception("C: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],1000.0,1e-4): raise Exception("K: incorrect value")
    for i in range(1,10):
        if not math.isnan(K[0,i]): raise Exception("K: incorrect nan value")
    if not isclose(K[1,0],10.0,1e-4): raise Exception("K: incorrect value")
    for i in range(1,10):
        if not math.isnan(K[1,i]): raise Exception("K: incorrect nan value")
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if R[0] != 0: raise Exception("R: incorrect value")
    if R[1] != 1: raise Exception("R: incorrect value")
    if S.shape != (1,2): raise Exception("S: incorrect shape")
    if S[0,0] != 1: raise Exception("S: incorrect value")
    if S[0,1] != -1: raise Exception("S: incorrect value")
    fp.close()

In [7]:
def test_second_order_reversible(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if nr != 2: raise Exception("numberReactions: incorrect value")
    if ns != 3: raise Exception("numberSpecies: incorrect value")
    if D.shape != (3,2): raise Exception("D: incorrect shape",D.shape)
    if D[0,0] != 1: raise Exception("D: incorrect value")
    if D[0,1] != 0: raise Exception("D: incorrect value")
    if D[1,0] != 2: raise Exception("D: incorrect value")
    if D[1,1] != 0: raise Exception("D: incorrect value")
    if D[2,0] != 0: raise Exception("D: incorrect value")
    if D[2,1] != 1: raise Exception("D: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if C[0] != 1000: raise Exception("C: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],0.9,1e-4): raise Exception("K: incorrect value")
    for i in range(1,10):
        if not math.isnan(K[0,i]): raise Exception("K: incorrect nan value")
    if not isclose(K[1,0],10.0,1e-4): raise Exception("K: incorrect value")
    for i in range(1,10):
        if not math.isnan(K[1,i]): raise Exception("K: incorrect nan value")
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if R[0] != 2: raise Exception("R: incorrect value")
    if R[1] != 1: raise Exception("R: incorrect value")
    if S.shape != (3,2): raise Exception("S: incorrect shape")
    if S[0,0] != -1: raise Exception("S: incorrect value")
    if S[0,1] != 1: raise Exception("S: incorrect value")
    if S[1,0] != -1: raise Exception("S: incorrect value")
    if S[1,1] != 1: raise Exception("S: incorrect value")
    if S[2,0] != 1: raise Exception("S: incorrect value")
    if S[2,1] != -1: raise Exception("S: incorrect value")
    fp.close()

In [8]:
def test_self_regulating_gene(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if nr != 2: raise Exception("numberReactions: incorrect value")
    if ns != 1: raise Exception("numberSpecies: incorrect value")
    if D.shape != (ns,nr): raise Exception("D: incorrect shape",D.shape)
    if D[0,0] != 1: raise Exception("D: incorrect value")
    if D[0,1] != 1: raise Exception("D: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if C[0] != 100: raise Exception("C: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],50,1e-4): raise Exception("K: incorrect value")
    if not isclose(K[0,1],50,1e-4): raise Exception("K: incorrect value")
    if not isclose(K[0,2],1000-50,1e-4): raise Exception("K: incorrect value")
    if not isclose(K[0,3],2,1e-4): raise Exception("K: incorrect value")
    for i in range(4,10):
        if not math.isnan(K[0,i]): raise Exception("K: incorrect nan value")
    if not isclose(K[1,0],10.0,1e-4): raise Exception("K: incorrect value")
    for i in range(1,10):
        if not math.isnan(K[1,i]): raise Exception("K: incorrect nan value")
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if R[0] != 8008: raise Exception("R: incorrect value")
    if R[1] != 1: raise Exception("R: incorrect value")
    if S.shape != (1,2): raise Exception("S: incorrect shape")
    if S[0,0] != 1: raise Exception("S: incorrect value")
    if S[0,1] != -1: raise Exception("S: incorrect value")
    fp.close()

### Run the tests.

In [9]:
testMethods = {"dimerization_reversible_mol_min": test_dimerization_reversible,
              "dimerization_reversible_mol_sec": test_dimerization_reversible,
              "dimerization_reversible_particle_min": test_dimerization_reversible,
              "dimerization_reversible_particle_sec": test_dimerization_reversible,
              "first_order_birth_death_mol_min": test_first_order_birth_death,
              "first_order_birth_death_mol_sec": test_first_order_birth_death,
              "first_order_birth_death_particle_min": test_first_order_birth_death,
              "first_order_birth_death_particle_sec": test_first_order_birth_death,
              "second_order_reversible_mol_min": test_second_order_reversible,
              "second_order_reversible_mol_sec": test_second_order_reversible,
              "second_order_reversible_particle_min": test_second_order_reversible,
              "second_order_reversible_particle_sec": test_second_order_reversible,
              "self_regulating_gene_particle_sec_1": test_self_regulating_gene,
              "self_regulating_gene_particle_sec": test_self_regulating_gene,
              "self_regulating_gene_particle_min": test_self_regulating_gene}
for testName in testNames:
    try:
        testOutputFilename="tmp/%s.lm"%(testName)
        testMethods[testName](testOutputFilename)
    except Exception as e:
        print "%-60s : FAILED with:"%("["+testName+"]"),e
    except:
        print "%-60s : FAILED with: Unknown exception"%("["+testName+"]")
    else:
        print "%-60s : passed."%("["+testName+"]")

[yeast_calcium_homeostasis]                                  : FAILED with: 'yeast_calcium_homeostasis'


In [10]:
%%bash
rm -rf tmp